# Colour Magnitude Diagram

```{warning}
This page is still under construction. It will be ready in time for the S263 data reduction sessions.
```

In this notebook, we read the photometric catalogs of the science frames, and draw a colour magnitude diagram.

You can download this page as a {download}`jupyter notebook <./CMD.ipynb>` file.

```{note}
In case you have been working on an AIfA lab room computer until this point, you might be interested in moving to a personal laptop (for example) for these last steps.
All that we'll need from here on is the content of the `PHOTOMETRY` directory (including the reference catalog). This is a relatively small amount of data (compared to the raw or pre-reduced images), so you could consider to copy just these files to your own computer. The CPU requirements for these last steps will also be negligible. 
```

In [ ]:
import dataredconfig

import numpy as np
import astropy
import astropy.table
import astropy.visualization
from astropy import units as u

%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt

## Gathering and structuring the data

We start by reading the relevant catalogs as a list of tables.

In [ ]:
# Where the photometry catalogs are:
photometry_dir = dataredconfig.work_dir / "PHOTOMETRY"

object_to_process = "M 37"

catalog_filepaths = sorted(list(photometry_dir.glob('*.fits')))
catalogs = []

for catalog_filepath in catalog_filepaths:
    
    catalog = astropy.table.Table.read(catalog_filepath)

    # We select the photometric catalogs of our object:  
    if "OBJECT" in catalog.meta:
        if catalog.meta["OBJECT"] == object_to_process:
            print(f"{catalog_filepath} : {catalog.meta}")
            catalogs.append(catalog)

print(f"Collected {len(catalogs)} catalogs.")

Now we "merge" all these separate tables into a particular structure that will be very useful.

 * Measurements done with the same filter are stacked in "depth", i.e., grouped in 2D-columns.
 * Finally all filters are stacked "horizontally", i.e., as separate columns in one single table.

In [ ]:

filter_names = ("g", "r", "i")
filter_catalogs = []

for filter_name in filter_names:
    #print(f"Starting filter {filter_name}")
    this_filter_catalogs = [catalog for catalog in catalogs if catalog.meta["FILTER"] == filter_name]
    this_filter_catalog = astropy.table.dstack(this_filter_catalogs, join_type="exact", metadata_conflicts="silent")
    this_filter_catalog.name = filter_name
    
    filter_catalogs.append(this_filter_catalog)

# And we stack these catalogs horizontally into one single table:
cat = astropy.table.hstack(filter_catalogs,
        join_type="exact", metadata_conflicts="silent",
        table_names=filter_names, uniq_col_name='{col_name}_{table_name}'
        )

# The columns are now the following:
print("Columns of cat:")
for colname in cat.colnames:
    print(f"{colname}: shape {cat[colname].shape}")


Note how the filter name is now appended to the column names.
These columns have two dimension: the first index identifies the source (i.e., star), and the second index runs over the different exposures.

In [ ]:

# We add a column with the separation between each star and the cluster center.
# To get the position, we need to read the reference catalog:

ref_catalog = astropy.table.Table.read(photometry_dir / f"ref_catalog_{object_to_process}.fits")
assert len(ref_catalog) == len(cat) # Just a check that these are indeed of same length
    
cat["sky_pos"] = ref_catalog["sky_centroid_win"]
target_center_pos = astropy.coordinates.SkyCoord(cat.meta["RA"]*u.deg, cat.meta["DEC"]*u.deg)
cat["separation"] = target_center_pos.separation(cat["sky_pos"])


### A quick CMD in instrumental magnitudes

As an example, we compute a median instrumental magnitude in each filter (where the median is taken over the different exposures).
This can certainly be improved, feel invited to experiment.

In [ ]:

# We take the median accross "axis=1", that is the second index.
# The first index (axis=0) identifies the different stars.
cat["instr_mag_g"] = np.median(-2.5 * np.log10(cat["sum_4_g"].value), axis=1)
cat["instr_mag_r"] = np.median(-2.5 * np.log10(cat["sum_4_r"].value), axis=1)
cat["instr_mag_i"] = np.median(-2.5 * np.log10(cat["sum_4_i"].value), axis=1)


plt.figure()
plt.scatter(
    cat["instr_mag_g"] - cat["instr_mag_i"],
    cat["instr_mag_r"],
    s = 5,
    c = cat["separation"].value # color represents separation between star and cluster center
)
plt.gca().invert_yaxis()
plt.colorbar(label=f"Separation to target center in {cat['separation'].unit}")
plt.xlabel("Instrumental mag g - i")
plt.ylabel("Instrumental mag r")
plt.title(object_to_process)
plt.show()



## Magnitude calibration


```{admonition} Question
Example of a question.
```

## Discussion of the CMD

```{admonition} Question
Can you estimate the age and distance of your cluster, by comparing its calibrated CMD to the isochrones. 
```